In [1]:
import numpy as np
import os
from utils.utilities import *
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import tensorflow as tf

## Load data

In [3]:
DATASET_PATH = "./dataset/UCI HAR Dataset/"

In [4]:
INPUT_SIGNAL_TYPES = [
    "body_acc_x_",
    "body_acc_y_",
    "body_acc_z_",
    "body_gyro_x_",
    "body_gyro_y_",
    "body_gyro_z_",
    "total_acc_x_",
    "total_acc_y_",
    "total_acc_z_"
]

In [5]:
def load_X(X_signals_paths):
    X_signals = []
    
    for signal_type_path in X_signals_paths:
        with open(signal_type_path, "r") as f:
            X_signals.append(
                [np.array(serie, dtype=np.float32)
                    for serie in [row.replace('  ', ' ').strip().split(' ') for row in f]]
            )
    return np.transpose(X_signals, (1, 2, 0))

def load_y(y_path):

    # Read dataset from disk, dealing with text file's syntax
    with open(y_path, "r") as f:
        y = np.array(
            [elem for elem in [
                row.replace('  ', ' ').strip().split(' ') for row in f
            ]], 
            dtype=np.int32
        )
            
    y = y.reshape(-1,)
    # Substract 1 to each output class for friendly 0-based indexing 
    return y - 1

In [6]:
X_train_signals_paths = [
    DATASET_PATH + "train/Inertial Signals/" + signal + "train.txt" for signal in INPUT_SIGNAL_TYPES
]
X_test_signals_paths = [
    DATASET_PATH + "test/Inertial Signals/" + signal + "test.txt" for signal in INPUT_SIGNAL_TYPES
]

In [7]:
har_X_train = load_X(X_train_signals_paths)
har_X_test = load_X(X_test_signals_paths)

In [8]:
y_train_path = DATASET_PATH + "train/y_train.txt"
y_test_path = DATASET_PATH + "test/y_test.txt"

In [9]:
har_y_train = load_y(y_train_path)
har_y_test = load_y(y_test_path)

## Prepare data

In [10]:
X_train, X_valid, y_train, y_valid = train_test_split(har_X_train, har_y_train, stratify=har_y_train, random_state=42)

In [11]:
X_train.shape, y_train.shape

((5514, 128, 9), (5514,))

In [12]:
X_valid.shape

(1838, 128, 9)

In [48]:
har_X_test.shape

(2947, 128, 9)

# Build CNN model

## Hyperparameters

In [14]:
batch_size = 600
learning_rate = 0.001
n_epochs = 200

# outputs
n_classes = 6

seq_len = 128
n_channels = 9

In [33]:
n_batches = X_train.shape[0] // batch_size

## first CNN model

In [35]:
tf.reset_default_graph()

X = tf.placeholder(tf.float32, (None, seq_len, n_channels), name="X")
y = tf.placeholder(tf.int32, (None), name="y")

# is in training phase
training = tf.placeholder_with_default(False, shape=(), name="training")


In [36]:
with tf.name_scope("cnn"):
    
    # Convolutional layers
    conv1 = tf.layers.conv1d(X, 
                             filters=32, 
                             kernel_size=2, 
                             strides=1,
                             padding="same",
                             activation=tf.nn.relu)

    conv2 = tf.layers.conv1d(conv1,
                             filters=64, 
                             kernel_size=2,
                             strides=1,
                             padding="same",
                             activation=tf.nn.relu)

    
    last_shape = conv2.get_shape().as_list()
    print(last_shape)
    # Fully Connected layers
    flat = tf.reshape(conv2, (-1, last_shape[1] * last_shape[2]))
   
    fc1 = tf.layers.dense(flat, 100, activation=tf.nn.relu)
    fc2 = tf.layers.dense(fc1, 100, activation=tf.nn.relu)
    
    logits = tf.layers.dense(fc2, n_classes)
    
    

[None, 128, 64]


In [37]:
logits.get_shape()

TensorShape([Dimension(None), Dimension(6)])

In [38]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y)
    loss = tf.reduce_mean(xentropy, name="loss")

In [39]:
with tf.name_scope("train"):
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    training_op = optimizer.minimize(loss)

In [40]:
with tf.name_scope("eval"):
    #correct = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

In [41]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [43]:
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        shuffled_idx = np.random.permutation(X_train.shape[0])
        X_random = X_train[shuffled_idx]
        y_random = y_train[shuffled_idx]
        for i in range(n_batches):
            X_batch = X_random[i * batch_size:(i+1) * batch_size]
            y_batch = y_random[i * batch_size:(i+1) * batch_size]
            train_acc =  sess.run([training_op], feed_dict={X: X_batch, y: y_batch, training: True})
        valid_acc = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        
        if epoch % 10 == 0:
            print("Epoch: ", epoch, "Valid accuracy: ", valid_acc)
    saver.save(sess, "./model/har_cnn_model.ckpt")

Epoch:  0 Valid accuracy:  0.710555
Epoch:  10 Valid accuracy:  0.939608
Epoch:  20 Valid accuracy:  0.95593
Epoch:  30 Valid accuracy:  0.960827
Epoch:  40 Valid accuracy:  0.959195
Epoch:  50 Valid accuracy:  0.966812
Epoch:  60 Valid accuracy:  0.964635
Epoch:  70 Valid accuracy:  0.972796
Epoch:  80 Valid accuracy:  0.970076
Epoch:  90 Valid accuracy:  0.972796
Epoch:  100 Valid accuracy:  0.978781
Epoch:  110 Valid accuracy:  0.976605
Epoch:  120 Valid accuracy:  0.979325
Epoch:  130 Valid accuracy:  0.958107
Epoch:  140 Valid accuracy:  0.975517
Epoch:  150 Valid accuracy:  0.979869
Epoch:  160 Valid accuracy:  0.980958
Epoch:  170 Valid accuracy:  0.978781
Epoch:  180 Valid accuracy:  0.976061
Epoch:  190 Valid accuracy:  0.981502


In [45]:
with tf.Session() as sess:
    saver.restore(sess, "./model/har_cnn_model.ckpt")
    test_acc = accuracy.eval(feed_dict={X: har_X_test, y: har_y_test})
    print("Test Accuracy: ", test_acc)

INFO:tensorflow:Restoring parameters from ./model/har_cnn_model.ckpt
Test Accuracy:  0.895487


## CNN model

In [49]:
tf.reset_default_graph()

X = tf.placeholder(tf.float32, (None, seq_len, n_channels), name="X")
y = tf.placeholder(tf.int32, (None), name="y")

# is in training phase
training = tf.placeholder_with_default(False, shape=(), name="training")

# learning rate exponential decay
global_step = tf.Variable(0, trainable=False)
starter_learning_rate = 0.01
learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
                                           100, 0.96, staircase=True)

In [50]:
with tf.name_scope("cnn"):
    
    # Convolutional layers
    conv1 = tf.layers.conv1d(X, 
                             filters=32, 
                             kernel_size=2, 
                             strides=1,
                             padding="same",
                             activation=tf.nn.relu)

    conv2 = tf.layers.conv1d(conv1,
                             filters=64, 
                             kernel_size=2,
                             strides=1,
                             padding="same",
                             activation=tf.nn.relu)

    conv3 = tf.layers.conv1d(conv1,
                             filters=96, 
                             kernel_size=2,
                             strides=1,
                             padding="same",
                             activation=tf.nn.relu)
    
    last_shape = conv3.get_shape().as_list()
    print(last_shape)
    # Fully Connected layers
    flat = tf.reshape(conv3, (-1, last_shape[1] * last_shape[2]))
    #drop = tf.nn.dropout(flat, keep_prob=1.0, training=training)
    
    fc1 = tf.layers.dense(flat, 100)
    bn_fc1 = tf.layers.batch_normalization(fc1, training=training)
    bn_fc1_act = tf.nn.relu(bn_fc1)
    
    fc2 = tf.layers.dense(bn_fc1_act, 100)
    bn_fc2 = tf.layers.batch_normalization(fc2, training=training)
    bn_fc2_act = tf.nn.relu(bn_fc2)
    
    logits_before_bn = tf.layers.dense(bn_fc2_act, n_classes)
    logits = tf.layers.batch_normalization(logits_before_bn, training=training)
    #logits = tf.layers.dense(bn_fc2_act, n_classes)
    
    

[None, 128, 96]


In [51]:
logits.get_shape()

TensorShape([Dimension(None), Dimension(6)])

In [52]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y)
    loss = tf.reduce_mean(xentropy, name="loss")

In [53]:
with tf.name_scope("train"):
    extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
    with tf.control_dependencies(extra_update_ops):
        optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
        training_op = optimizer.minimize(loss, global_step=global_step)

In [54]:
with tf.name_scope("eval"):
    #correct = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

In [55]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [56]:
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        shuffled_idx = np.random.permutation(X_train.shape[0])
        X_random = X_train[shuffled_idx]
        y_random = y_train[shuffled_idx]
        for i in range(n_batches):
            X_batch = X_random[i * batch_size:(i+1) * batch_size]
            y_batch = y_random[i * batch_size:(i+1) * batch_size]
            train_acc =  sess.run([training_op], feed_dict={X: X_batch, y: y_batch, training: True})
        valid_acc = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        
        if epoch % 10 == 0:
            print("Epoch: ", epoch,"Learning rate:", sess.run(optimizer._lr), "Valid accuracy: ", valid_acc)
    saver.save(sess, "./model/har_cnn_model.ckpt")

Epoch:  0 Learning rate: 0.01 Valid accuracy:  0.145811
Epoch:  10 Learning rate: 0.01 Valid accuracy:  0.570729
Epoch:  20 Learning rate: 0.0096 Valid accuracy:  0.846572
Epoch:  30 Learning rate: 0.009216 Valid accuracy:  0.936888
Epoch:  40 Learning rate: 0.00884736 Valid accuracy:  0.957018
Epoch:  50 Learning rate: 0.00849346 Valid accuracy:  0.972796
Epoch:  60 Learning rate: 0.00815373 Valid accuracy:  0.97062
Epoch:  70 Learning rate: 0.00782758 Valid accuracy:  0.95321
Epoch:  80 Learning rate: 0.00751447 Valid accuracy:  0.968988
Epoch:  90 Learning rate: 0.00721389 Valid accuracy:  0.967356
Epoch:  100 Learning rate: 0.00692534 Valid accuracy:  0.966812
Epoch:  110 Learning rate: 0.00692534 Valid accuracy:  0.964635
Epoch:  120 Learning rate: 0.00664832 Valid accuracy:  0.980413
Epoch:  130 Learning rate: 0.00638239 Valid accuracy:  0.980413
Epoch:  140 Learning rate: 0.0061271 Valid accuracy:  0.979325
Epoch:  150 Learning rate: 0.00588201 Valid accuracy:  0.981502
Epoch:  

In [58]:
with tf.Session() as sess:
    saver.restore(sess, "./model/har_cnn_model.ckpt")
    test_acc = accuracy.eval(feed_dict={X: har_X_test, y: har_y_test})
    print("Test Accuracy: ", test_acc)

INFO:tensorflow:Restoring parameters from ./model/har_cnn_model.ckpt
Test Accuracy:  0.890397
